# Import

In [1]:
# extensions for jupyter notebook
#reload install packages w/o restartting kernel
%load_ext autoreload 
%autoreload 2
#plots display in the cell
%matplotlib inline
%config InlineBackend.figure_format = 'png'
#faster autocompletion
%config IPCompleter.use_jedi = False 

# data analysis packages
import pandas as pd
import numpy as np

# plotting packages & params for figures
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('ticks')
plt.rcParams['pdf.fonttype'] = 'truetype'
plt.rcParams['svg.fonttype'] = 'none'
plt.rcParams['figure.dpi'] = 120

import warnings
warnings.filterwarnings('ignore')

# Data

Prepare you sample according to the [guidelines](https://github.com/BostonGene/Immune_Escape/blob/main/DATA_AND_QC.md) (take required transcripts, perform QC), then load: 

In [2]:
from functions.utils import read_dataset

In [3]:
expression = read_dataset('test_data/QC_example_SKCM_HugoMAPK2_GSE75313/output/expressions_refined.tsv.gz')
expression.head()

,SRX1446796,SRX1446797,SRX1446798,SRX1446799,SRX1446800,SRX1446801,SRX1446802,SRX1446803,SRX1446804,SRX1446805,SRX1446808,SRX1446809,SRX1446810,SRX1446811,SRX1446812,SRX3173615,SRX3173616,SRX3173617
Gene,,,,,,,,,,,,,,,,,,
A1BG,0.000000,0.000000,0.073078,0.046100,0.000000,0.000000,0.031377,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.360252,0.125111,0.000000,0.185589
A1CF,0.000000,0.000000,0.016818,0.032895,0.018860,0.030272,0.020603,0.018773,0.015271,0.003989,0.027835,0.018157,0.042922,0.018590,0.000000,0.081888,0.052673,0.071418
A2M,8.315325,8.688029,7.213757,8.189954,9.775147,9.202142,8.738409,8.469137,9.093775,10.379255,8.937630,9.948894,8.181539,8.152129,8.168005,7.846306,8.958495,9.082054
A2ML1,1.939184,3.383156,5.002732,5.324455,0.676129,0.108234,1.979346,1.943242,0.596630,0.193554,0.180695,0.083906,0.308971,0.130943,0.519393,3.735812,0.518568,4.607811
A3GALT2,0.000000,0.080243,0.000000,0.000000,0.800559,0.293649,0.000000,0.029331,0.000000,0.000000,0.000000,0.705637,0.000000,0.061043,0.476854,0.786077,0.556590,0.659706


## Load metacohort for mapping

In [4]:
raw_features = read_dataset('data/unscaled_features.tsv')

# Calculate features

Load FGES and PROGENy coefficients; calculate features.

In [5]:
import pickle

from functions.feature_calc import gmt_genes_alt_names, ssgsea_formula, run_progeny
from functions.utils import median_scale

print(pickle.format_version)

4.0


In [6]:
# Load gmt and progeny coefficients from pickle

with open('model/IE_clusters.pickle', 'rb') as f:
    ie_dict = pickle.load(f)

gmt = ie_dict['gmt_dict']
gmt = gmt_genes_alt_names(gmt, expression.index, verbose=True)

prog_coeffs = ie_dict['progeny_coeffs'].reset_index()

Matched: 356
Trying to find new names for 1 genes in 19706 known


1 input query terms found dup hits:	[('TRBC1', 2)]


1 genes were not converted


In [7]:
# Calculate ssgsea
ssgsea_df = ssgsea_formula(expression.T, gmt)

# Calculate progeny
progen_df = run_progeny(expression, prog_coeffs=prog_coeffs).T

In [8]:
#Concat and scale
features_df = pd.concat([ssgsea_df, progen_df], axis=1)[ie_dict['X'].columns]

features_df.head()

,Metabolic suppression of CTL,B cells,M1 cytokines,Treg cells,pDC,T cells,TLS_NL,Anti-tumor chemokines,Lymphoid_checkpoints,NK cells,...,Adipocytes,EGFR,MAPK,Hypoxia,Hypoxia_factors,Glycolysis,PI3K,Autophagy,Acidosis,Proliferation_rate
SRX1446796,3985.709952,-3884.138774,-1648.754375,-1769.612829,-79.245004,460.102859,1271.504103,2650.380093,-2279.968697,-2938.296899,...,-1026.297588,239.865412,719.737279,2919.926211,4801.965360,8145.301327,-1808.919072,5591.446739,3654.723778,4841.753306
SRX1446797,5174.958890,-3679.885859,-503.270988,-1323.023632,-947.570186,1571.705614,1366.566097,2471.748880,-2471.442183,-1914.812690,...,-1053.543692,68.943058,390.370477,2940.539383,5350.730203,7996.579165,-1717.763551,6263.916348,2582.211411,1351.137509
SRX1446798,2869.289496,-5196.271912,-2234.337666,-946.301099,-1680.457667,1430.556387,1499.660210,1689.882960,-2970.227740,-3330.854164,...,-3270.155298,207.712098,676.063867,3311.090882,5740.752062,8116.339947,-1850.359166,5676.118638,4725.338139,4604.241679
SRX1446799,3467.957320,-3947.158683,-3035.724010,-749.844071,-2566.339720,1142.469852,33.260150,744.416670,-4088.191395,-3783.843105,...,-1505.496207,22.902882,460.304813,3251.027969,6181.541590,7762.469580,-1682.728968,6147.326850,2704.519121,1936.318944
SRX1446800,2969.759781,-2739.281849,443.245442,-2955.165535,-1287.810252,-1498.351958,1581.599926,2986.897478,-3513.056746,-3056.053696,...,-2488.350161,127.799120,840.808627,3472.956247,5236.563707,7820.668200,-1991.965620,6056.842810,5578.752045,2666.941984


# Find reference, scale and classify

Load model, map samples to metacohort, scale and classify

In [9]:
from functions.utils import predict_batch

In [10]:
# Progress bar visualization
from tqdm import tqdm

In [18]:
# Custom function to calculate predictions

def get_batch_and_predicitons_for_samples(sample_df, ref_df, batch, k=100):
    batch_pred = {}
    probas = {}
    
    for name, ser in tqdm(sample_df.iterrows()):
        
        ser = ser[(~ser.isna())] # Can't process NaN features
        
        X_ref = ref_df[ser.index].dropna() # Same set of features; be consious, classifier only take full set of features!
    
        ident_batch = predict_batch(ser, X_ref, batch, k=k) #Higher k will produce better results
        batch_pred[name] = ident_batch
    
        ref_coh = X_ref[batch==ident_batch]
        
        scaled_features = median_scale(pd.concat([ref_coh,ser.to_frame().T])) #concat with identified cohort and scale
        scaled_features = scaled_features.loc[name]
        
        prob = model.predict_proba(scaled_features.to_frame().T)
        probas[name] = list(prob[0])
    
    batch_pred = pd.Series(batch_pred)
    probas = pd.DataFrame(probas)
    probas.index = model.classes_ 
    
    probas.loc['Unclassified'] = 0
    probas.loc['Unclassified',probas.columns[probas[probas>0.47].isna().all()]] = 1 # Set threshold
    
    class_predict = probas.idxmax()

    return batch_pred, probas, class_predict

In [19]:
# Load pre-treained model
with open('model/ovr_knn_calibrated.pickle', 'rb') as f:
    model = pickle.load(f)

In [20]:
# Define variables
sample_df = features_df
ref_df = raw_features

batch = ref_df['Batch']
ref_df = ref_df.drop(columns='Batch')

# Run prediction
batch_pred, probas, class_predict = get_batch_and_predicitons_for_samples(sample_df, ref_df, batch, k=100)

18it [00:01, 10.37it/s]


In [21]:
class_predict.value_counts()

Immune-Enriched, Fibrotic           6
Unclassified                        3
Highly Immune-Enriched, Inflamed    2
Fibrotic, Angiogenic, Myeloid       2
Fibrotic, Hypoxic                   2
Lymphoid-Cell-Enriched              1
Faintly Infiltrated, Angiogenic     1
Immune-Enriched, Hypoxic            1
dtype: int64

In [22]:
batch_pred.value_counts()

TCGA-SKCM             6
TCGA-BRCA_LumiNorm    5
TCGA-KIRC             3
TCGA-PAAD             1
TCGA-LUAD             1
TCGA-HNE_SCC          1
TCGA-LUSC             1
dtype: int64

In [15]:
class_predict.to_csv('test_data/QC_example_SKCM_HugoMAPK2_GSE75313/output/class_prediction_identified_batch_scaling.tsv', sep='\t')
batch_pred.to_csv('test_data/QC_example_SKCM_HugoMAPK2_GSE75313/output/identified_batch.tsv', sep='\t')